### 물체 인식률 실험

In [1]:
import open3d as o3d
import numpy as np
import os
import sys
import open3d as o3d
import cv2
import copy
import subprocess
import matplotlib.pyplot as plt
# os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))
# sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/milestone_202110'))
# from scripts.milestone_202110.utils.detection_util import *
from util.multiICP import *
from test_util import *
from demo_utils.environment import *
# from utils.detection_util import *

In [2]:
from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.geometry.builder.scene_builder import SceneBuilder

In [3]:
import PIL
from cv2 import aruco
import matplotlib as mpl
import pandas as pd

aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
# fig = plt.figure()
# nx = 2
# ny = 2
# for i in range(1, nx*ny+1):
#     ax = fig.add_subplot(ny,nx, i)
#     img = aruco.drawMarker(aruco_dict,i+229, 800)
#     plt.imshow(img, cmap = mpl.cm.gray, interpolation = "nearest")
#     ax.axis("off")
# plt.savefig("target_markers.pdf")

# # Save Aruco Marker Image
# for i in range(6):
#     idx = i+230
#     img = aruco.drawMarker(aruco_dict, idx, 800)
#     cv2.imwrite("marker_{}.png".format(idx), img)

In [4]:
cameraMatrix = np.array([[899.05322266,   0.        , 654.88366699],
                          [  0.        , 899.21044922, 352.92956543],
                          [  0.        ,   0.        ,   1.        ]])
distCoeffs = np.array([ 0.15870179, -0.46903715, -0.0014261 ,  0.00066797,  0.41631824])

obj_name_off = '--obj_name='
img_num_off = '--img_num='
confidence_off = '--confidence='

### Mini_table

In [5]:
path = './save_img/mini_table'

In [8]:
# camera-streaming
cam_streaming(path)

./save_img/mini_table not empty. Overwrite? (y/n) : y
Saved color + depth image 0
Saved color + depth image 1
Saved color + depth image 2
Saved color + depth image 3
Saved color + depth image 4
Saved color + depth image 5
Saved color + depth image 6
Saved color + depth image 7
Saved color + depth image 8
Saved color + depth image 9
Saved color + depth image 10
Saved color + depth image 11
Saved color + depth image 12
Saved color + depth image 13
Saved color + depth image 14
Saved color + depth image 15
Saved color + depth image 16
Saved color + depth image 17
Saved color + depth image 18
Saved color + depth image 19
Stop the streaming and Save RGBD images



In [6]:
confidence = 0.01
obj_name_arg = obj_name_off + 'mini_table'
img_num_arg = img_num_off + str(20)
confidence_arg = confidence_off + str(confidence)
result = subprocess.check_output(['python3',
                    os.getcwd() + '/test_seg_multi.py',
                    obj_name_arg, img_num_arg, confidence_arg])
result

'apex is not installed\napex is not installed\napex is not installed\napex is not installed\nUse load_from_local loader\nNot segmented at 2 image\nNot segmented at 3 image\nNot segmented at 4 image\nNot segmented at 5 image\nNot segmented at 7 image\nNot segmented at 8 image\nNot segmented at 15 image\nNot segmented at 18 image\nSegmentation Finish!\n'

In [7]:
mini_table_iou = []

In [8]:
for i in range(20):
    color_path = os.path.join(path, 'color_segmented/color_mask_{}.jpg'.format(i))
    depth_path = os.path.join(path, 'depth_segmented/depth_mask_{}.png'.format(i))
    cdp_masked = load_cdp(color_path, depth_path)
    if cdp_masked.color is None:
        mini_table_iou.append(0.0)
        pass
    else:

        icp_table = MultiICP(model='model_CAD/mini_table.STL',
                         Toff=SE3(np.identity(3),(0,0,0)), scale=[1e-3,1e-3,1e-3])
        icp_table.add_image(cdp_masked, Tc=None)

    #     Tguess = SE3(Rot_axis(1, 2*np.pi/3), (-0.15,0,0.7))
        Tguess = icp_table.get_initial_by_center(Rot_axis(1, 2*np.pi/3), offset=(-0.27,0.2,0))
        T_co1, fitness1 = icp_table.compute_ICP(To=Tguess, thres=0.2, visualize=False)
    #     T_co2, fitness2 = icp_table.compute_ICP(To=None, thres=0.3, visualize=False)

    #     if fitness1 > fitness2:
    #         T_co = T_co1
    #     else:
    #         T_co = T_co2
        T_co = T_co1
        color_image = cv2.imread(os.path.join(path, 'color/color_{}.jpg').format(i))
        objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
        # img_out = aruco_map.draw_objects(color_image, objectPose_dict, corner_dict, cameraMatrix, distCoeffs)
        # plt.imshow(img_out[:,:,[2,1,0]])

        T_cm = objectPose_dict['test_table']
        T_mo = SE3(Rot_axis(3, -np.pi/2), (-0.09, 0.03, -0.265))
        T_co_gt = np.matmul(T_cm, T_mo)
        T_co, T_co_gt

        T_off = np.matmul(SE3_inv(T_co_gt), T_co)
        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=(0,0,0))

        mesh = o3d.io.read_triangle_mesh('model_CAD/mini_table.STL')
        mesh_gt = o3d.io.read_triangle_mesh('model_CAD/mini_table.STL')
        mesh.vertices = o3d.utility.Vector3dVector(
            np.asarray(mesh.vertices) * np.array([1 / 1000.0, 1 / 1000.0, 1 / 1000.0]))
        mesh_gt.vertices = o3d.utility.Vector3dVector(
            np.asarray(mesh_gt.vertices) * np.array([1 / 1000.0, 1 / 1000.0, 1 / 1000.0]))
        mesh.transform(T_off)
        # mesh_gt.transform(T_co_gt)
        aabox = mesh.get_oriented_bounding_box()
        aabox_gt = mesh_gt.get_oriented_bounding_box()

    #     o3d.visualization.draw_geometries([mesh, aabox, mesh_gt, aabox_gt, origin])

        points = np.asarray(aabox.get_box_points())
        points_gt = np.asarray(aabox_gt.get_box_points())
        # points = points[points[:,2].argsort()[::-1]]
        # points_gt = points_gt[points_gt[:,2].argsort()[::-1]]

    #     iou_3d = get_3d_iou(aabox, aabox_gt, points, points_gt)
        iou_3d = get_iou_3d(aabox, aabox_gt, points, points_gt, T_off)
        mini_table_iou.append(iou_3d)

Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.167165e-02, and correspondence_set size of 104717
Access transformation to get result.
Transformation is:
[[ 0.96686427 -0.25528853  0.00111334 -0.18339585]
 [-0.17455864 -0.66428246 -0.72681364  0.27441229]
 [ 0.18628676  0.7025358  -0.6868338   0.75660706]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.896752154262214)
Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.467975e-02, and correspondence_set size of 113267
Access transformation to get result.
Transformation is:
[[ 0.97541196 -0.2157194  -0.04512933 -0.20075301]
 [-0.18291821 -0.6781941  -0.71175394  0.28577226]
 [ 0.12293268  0.70250828 -0.70097764  0.75386696]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.9513014887606263)
Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.125799e-02, and co

In [9]:
mini_table_iou

[0.896752154262214,
 0.9513014887606263,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9202219733958005,
 0.0,
 0.0,
 0.9733160380703114,
 0.9902412635280228,
 0.9721598847566282,
 0.994290563725255,
 0.9917169135538296,
 0.9947244680698691,
 0.0,
 0.8109630316664765,
 0.8489798533411471,
 0.0,
 0.9001537723317045]

In [21]:
np.mean(np.array(mini_table_iou))

0.9422625965513596

In [10]:
np.savetxt('./exp_data/mini_table_iou_result_{}.txt'.format(confidence), np.array(mini_table_iou))

### Washstand

In [11]:
path = './save_img/washstand'

In [ ]:
# camera-streaming
cam_streaming(path)

In [12]:
confidence = 0.01
obj_name_arg = obj_name_off + 'washstand'
img_num_arg = img_num_off + str(20)
confidence_arg = confidence_off + str(confidence)
result = subprocess.check_output(['python3',
                    os.getcwd() + '/test_seg_multi.py',
                    obj_name_arg, img_num_arg, confidence_arg])
result

'apex is not installed\napex is not installed\napex is not installed\napex is not installed\nUse load_from_local loader\nSegmentation Finish!\n'

In [13]:
washstand_iou = []

In [14]:
for j in range(20):
    color_path = os.path.join(path, 'color_segmented/color_mask_{}.jpg'.format(j))
    depth_path = os.path.join(path, 'depth_segmented/depth_mask_{}.png'.format(j))
    cdp_masked = load_cdp(color_path, depth_path)
    if cdp_masked.color is None:
        washstand_iou.append(0.0)
        pass
    else:
        icp_washstand = MultiICP(model='model_CAD/washstand.stl',
                         Toff=SE3(np.identity(3),(0,0,0)), scale=[1.12e-2,1.4e-2,1.1e-2])
        icp_washstand.add_image(cdp_masked, Tc=None)

        scale=[1.12e-2,1.4e-2,1.1e-2]
        wash_model = o3d.io.read_triangle_mesh('model_CAD/washstand.stl')
        wash_model.vertices = o3d.utility.Vector3dVector(
                        np.asarray(wash_model.vertices) * np.array([scale[0], scale[1], scale[2]]))

        wash_pcd = wash_model.sample_points_uniformly(int(len(np.asarray(icp_washstand.pcd.points))*0.7))
        points = np.asarray(wash_pcd.points)

        idx = []
        for i in range(len(points)):
            if points[i, 2] > 0.65:
                idx.append(i)

        pts = np.zeros((len(idx), 3))
        for i in range(len(idx)):
            pts[i] = points[idx[i]]

        cut_pcd = o3d.geometry.PointCloud()
        cut_pcd.points = o3d.utility.Vector3dVector(pts)
        icp_washstand.model_sampled = cut_pcd

        Tguess = icp_washstand.get_initial_by_center(Rot_axis(1, 2*np.pi/3), offset=(-0.05,0.6,0.6))
        T_co1, fitness1 = icp_washstand.compute_ICP(To=Tguess, thres=0.15, visualize=False)

        T_co = T_co1

        color_image = cv2.imread(os.path.join(path, 'color/color_{}.jpg'.format(j)))
        objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)
    #     img_out = aruco_map.draw_objects(color_image, objectPose_dict, corner_dict, cameraMatrix, distCoeffs)
    #     plt.imshow(img_out[:,:,[2,1,0]])

        T_cm = objectPose_dict['test_table']
        T_mo = SE3(Rot_axis(1, -np.pi/2), (0.275, -1.065, -0.01))
        T_co_gt = np.matmul(T_cm, T_mo)

        T_off = np.matmul(SE3_inv(T_co_gt), T_co)

        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=(0,0,0))
        mesh = o3d.io.read_triangle_mesh('model_CAD/washstand.stl')
        mesh_gt = o3d.io.read_triangle_mesh('model_CAD/washstand.stl')
        mesh.vertices = o3d.utility.Vector3dVector(
                        np.asarray(mesh.vertices) * np.array([scale[0], scale[1], scale[2]]))
        mesh_gt.vertices = o3d.utility.Vector3dVector(
                        np.asarray(mesh_gt.vertices) * np.array([scale[0], scale[1], scale[2]]))
        mesh.transform(T_off)
        # mesh_gt.transform(T_co_gt)
        # aabox = mesh.get_oriented_bounding_box()
        # aabox_gt = mesh_gt.get_oriented_bounding_box()
        aabox = mesh.get_axis_aligned_bounding_box()
        aabox_gt = mesh_gt.get_axis_aligned_bounding_box()

    #     o3d.visualization.draw_geometries([mesh, aabox, mesh_gt, aabox_gt, origin])
        points = np.asarray(aabox.get_box_points())
        points_gt = np.asarray(aabox_gt.get_box_points())
        # points = points[points[:,2].argsort()[::-1]]
        # points_gt = points_gt[points_gt[:,2].argsort()[::-1]]

    #     iou_3d = get_3d_iou(aabox, aabox_gt, points, points_gt)
        iou_3d = get_iou_3d(aabox, aabox_gt, points, points_gt, T_off)
        washstand_iou.append(iou_3d)

Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.125604e-02, and correspondence_set size of 12652
Access transformation to get result.
Transformation is:
[[ 0.95038004 -0.27033379 -0.15393969  0.23888653]
 [-0.17620091 -0.05995544 -0.98252662  0.90421657]
 [ 0.25638063  0.960898   -0.10461352  1.54507352]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.7185847641729005)
Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.187762e-02, and correspondence_set size of 15216
Access transformation to get result.
Transformation is:
[[ 0.92349135 -0.36726482 -0.11081644  0.44668905]
 [-0.15374362 -0.08967326 -0.98403332  0.86072645]
 [ 0.35146356  0.92578358 -0.1392771   1.50761193]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.723221103700546)
Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.287902e-02, and corr

registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.445504e-02, and correspondence_set size of 25889
Access transformation to get result.
Transformation is:
[[ 0.88500092 -0.46288023  0.05015232 -0.06316346]
 [ 0.01702182 -0.0754788  -0.9970021   0.87109157]
 [ 0.46527801  0.88320146 -0.05891974  1.0939489 ]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.7473727713787528)


In [15]:
washstand_iou

[0.7185847641729005,
 0.723221103700546,
 0.7108182004171711,
 0.7595604008095227,
 0.7550057488393838,
 0.727397636990485,
 0.7313745802352348,
 0.7108170072212904,
 0.7649310801345218,
 0.7748319318685264,
 0.7346059416012531,
 0.7227672487503441,
 0.7030151891792775,
 0.45601688007045066,
 0.4523973720158529,
 0.4973296503152455,
 0.7070291657523541,
 0.7560597040839362,
 0.7850391098935965,
 0.7473727713787528]

In [12]:
np.mean(np.array(washstand_iou))

0.6985115173653922

In [16]:
np.savetxt('./exp_data/washstand_iou_result_{}.txt'.format(confidence), np.array(washstand_iou))

### Bed & Closet

In [17]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(None, None))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                           None, root_on="kmb0_platform", 
                           specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, False])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)
import os
import sys
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))
from demo_utils.environment import add_cam, add_bed, add_closet, move_bed, move_closet

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
VIEW_LOC = [0]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
T_bc = viewpoint.get_tf(VIEW_POSE_EXT)
T_cb = SE3_inv(T_bc)
gscene.show_pose(VIEW_POSE_EXT)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [18]:
path = './save_img/bed'

In [4]:
# camera-streaming
cam_streaming(path)

./save_img/bed not empty. Overwrite? (y/n) : y
Saved color + depth image 0
Saved color + depth image 1
Saved color + depth image 2
Saved color + depth image 3
Saved color + depth image 4
Saved color + depth image 5
Saved color + depth image 6
Saved color + depth image 7
Saved color + depth image 8
Saved color + depth image 9
Saved color + depth image 10
Saved color + depth image 11
Saved color + depth image 12
Saved color + depth image 13
Saved color + depth image 14
Saved color + depth image 15
Saved color + depth image 16
Saved color + depth image 17
Saved color + depth image 18
Saved color + depth image 19
Saved color + depth image 20
Saved color + depth image 21
Saved color + depth image 22
Saved color + depth image 23
Saved color + depth image 24
Saved color + depth image 25
Saved color + depth image 26
Saved color + depth image 27
Saved color + depth image 28
Saved color + depth image 29
Saved color + depth image 30
Saved color + depth image 31
Saved color + depth image 32
Saved 

Saved color + depth image 279
Saved color + depth image 280
Saved color + depth image 281
Saved color + depth image 282
Saved color + depth image 283
Saved color + depth image 284
Saved color + depth image 285
Saved color + depth image 286
Saved color + depth image 287
Saved color + depth image 288
Saved color + depth image 289
Saved color + depth image 290
Saved color + depth image 291
Saved color + depth image 292
Saved color + depth image 293
Saved color + depth image 294
Saved color + depth image 295
Saved color + depth image 296
Saved color + depth image 297
Saved color + depth image 298
Saved color + depth image 299
Saved color + depth image 300
Saved color + depth image 301
Saved color + depth image 302
Saved color + depth image 303
Saved color + depth image 304
Saved color + depth image 305
Saved color + depth image 306
Saved color + depth image 307
Saved color + depth image 308
Saved color + depth image 309
Saved color + depth image 310
Saved color + depth image 311
Saved colo

Saved color + depth image 554
Saved color + depth image 555
Saved color + depth image 556
Saved color + depth image 557
Saved color + depth image 558
Saved color + depth image 559
Saved color + depth image 560
Saved color + depth image 561
Saved color + depth image 562
Saved color + depth image 563
Saved color + depth image 564
Saved color + depth image 565
Saved color + depth image 566
Saved color + depth image 567
Saved color + depth image 568
Saved color + depth image 569
Saved color + depth image 570
Saved color + depth image 571
Saved color + depth image 572
Saved color + depth image 573
Saved color + depth image 574
Saved color + depth image 575
Saved color + depth image 576
Saved color + depth image 577
Saved color + depth image 578
Saved color + depth image 579
Saved color + depth image 580
Saved color + depth image 581
Saved color + depth image 582
Saved color + depth image 583
Saved color + depth image 584
Saved color + depth image 585
Saved color + depth image 586
Saved colo

Saved color + depth image 830
Saved color + depth image 831
Saved color + depth image 832
Saved color + depth image 833
Saved color + depth image 834
Saved color + depth image 835
Saved color + depth image 836
Saved color + depth image 837
Saved color + depth image 838
Saved color + depth image 839
Saved color + depth image 840
Saved color + depth image 841
Saved color + depth image 842
Saved color + depth image 843
Saved color + depth image 844
Saved color + depth image 845
Saved color + depth image 846
Saved color + depth image 847
Saved color + depth image 848
Saved color + depth image 849
Saved color + depth image 850
Saved color + depth image 851
Saved color + depth image 852
Saved color + depth image 853
Saved color + depth image 854
Saved color + depth image 855
Saved color + depth image 856
Saved color + depth image 857
Saved color + depth image 858
Saved color + depth image 859
Saved color + depth image 860
Saved color + depth image 861
Saved color + depth image 862
Saved colo

Saved color + depth image 1100
Saved color + depth image 1101
Saved color + depth image 1102
Saved color + depth image 1103
Saved color + depth image 1104
Saved color + depth image 1105
Saved color + depth image 1106
Saved color + depth image 1107
Saved color + depth image 1108
Saved color + depth image 1109
Saved color + depth image 1110
Saved color + depth image 1111
Saved color + depth image 1112
Saved color + depth image 1113
Saved color + depth image 1114
Saved color + depth image 1115
Saved color + depth image 1116
Saved color + depth image 1117
Saved color + depth image 1118
Saved color + depth image 1119
Saved color + depth image 1120
Saved color + depth image 1121
Saved color + depth image 1122
Saved color + depth image 1123
Saved color + depth image 1124
Saved color + depth image 1125
Saved color + depth image 1126
Saved color + depth image 1127
Saved color + depth image 1128
Saved color + depth image 1129
Saved color + depth image 1130
Saved color + depth image 1131
Saved co

...

Saved color + depth image 12049
Saved color + depth image 12050
Saved color + depth image 12051
Saved color + depth image 12052
Saved color + depth image 12053
Saved color + depth image 12054
Saved color + depth image 12055
Saved color + depth image 12056
Saved color + depth image 12057
Saved color + depth image 12058
Saved color + depth image 12059
Saved color + depth image 12060
Saved color + depth image 12061
Saved color + depth image 12062
Saved color + depth image 12063
Saved color + depth image 12064
Saved color + depth image 12065
Saved color + depth image 12066
Saved color + depth image 12067
Saved color + depth image 12068
Saved color + depth image 12069
Saved color + depth image 12070
Saved color + depth image 12071
Saved color + depth image 12072
Saved color + depth image 12073
Saved color + depth image 12074
Saved color + depth image 12075
Saved color + depth image 12076
Saved color + depth image 12077
Saved color + depth image 12078
Saved color + depth image 12079
Saved co

Saved color + depth image 12311
Saved color + depth image 12312
Saved color + depth image 12313
Saved color + depth image 12314
Saved color + depth image 12315
Saved color + depth image 12316
Saved color + depth image 12317
Saved color + depth image 12318
Saved color + depth image 12319
Saved color + depth image 12320
Saved color + depth image 12321
Saved color + depth image 12322
Saved color + depth image 12323
Saved color + depth image 12324
Saved color + depth image 12325
Saved color + depth image 12326
Saved color + depth image 12327
Saved color + depth image 12328
Saved color + depth image 12329
Saved color + depth image 12330
Saved color + depth image 12331
Saved color + depth image 12332
Saved color + depth image 12333
Saved color + depth image 12334
Saved color + depth image 12335
Saved color + depth image 12336
Saved color + depth image 12337
Saved color + depth image 12338
Saved color + depth image 12339
Saved color + depth image 12340
Saved color + depth image 12341
Saved co

Saved color + depth image 12569
Saved color + depth image 12570
Saved color + depth image 12571
Saved color + depth image 12572
Saved color + depth image 12573
Saved color + depth image 12574
Saved color + depth image 12575
Saved color + depth image 12576
Saved color + depth image 12577
Saved color + depth image 12578
Saved color + depth image 12579
Saved color + depth image 12580
Saved color + depth image 12581
Saved color + depth image 12582
Saved color + depth image 12583
Saved color + depth image 12584
Saved color + depth image 12585
Saved color + depth image 12586
Saved color + depth image 12587
Saved color + depth image 12588
Saved color + depth image 12589
Saved color + depth image 12590
Saved color + depth image 12591
Saved color + depth image 12592
Saved color + depth image 12593
Saved color + depth image 12594
Saved color + depth image 12595
Saved color + depth image 12596
Saved color + depth image 12597
Saved color + depth image 12598
Saved color + depth image 12599
Saved co

Saved color + depth image 12826
Saved color + depth image 12827
Saved color + depth image 12828
Saved color + depth image 12829
Saved color + depth image 12830
Saved color + depth image 12831
Saved color + depth image 12832
Saved color + depth image 12833
Saved color + depth image 12834
Saved color + depth image 12835
Saved color + depth image 12836
Saved color + depth image 12837
Saved color + depth image 12838
Saved color + depth image 12839
Saved color + depth image 12840
Saved color + depth image 12841
Saved color + depth image 12842
Saved color + depth image 12843
Saved color + depth image 12844
Saved color + depth image 12845
Saved color + depth image 12846
Saved color + depth image 12847
Saved color + depth image 12848
Saved color + depth image 12849
Saved color + depth image 12850
Saved color + depth image 12851
Saved color + depth image 12852
Saved color + depth image 12853
Saved color + depth image 12854
Saved color + depth image 12855
Saved color + depth image 12856
Saved co

Saved color + depth image 13087
Saved color + depth image 13088
Saved color + depth image 13089
Saved color + depth image 13090
Saved color + depth image 13091
Saved color + depth image 13092
Saved color + depth image 13093
Saved color + depth image 13094
Saved color + depth image 13095
Saved color + depth image 13096
Saved color + depth image 13097
Saved color + depth image 13098
Saved color + depth image 13099
Saved color + depth image 13100
Saved color + depth image 13101
Saved color + depth image 13102
Saved color + depth image 13103
Saved color + depth image 13104
Saved color + depth image 13105
Saved color + depth image 13106
Saved color + depth image 13107
Saved color + depth image 13108
Saved color + depth image 13109
Saved color + depth image 13110
Saved color + depth image 13111
Saved color + depth image 13112
Saved color + depth image 13113
Saved color + depth image 13114
Saved color + depth image 13115
Saved color + depth image 13116
Saved color + depth image 13117
Saved co

In [19]:
confidence = 0.01
obj_name_arg = obj_name_off + 'bed'
img_num_arg = img_num_off + str(20)
confidence_arg = confidence_off + str(confidence)
result = subprocess.check_output(['python3',
                    os.getcwd() + '/test_seg_multi.py',
                    obj_name_arg, img_num_arg, confidence_arg])
result

'apex is not installed\napex is not installed\napex is not installed\napex is not installed\nUse load_from_local loader\nSegmentation Finish!\n'

In [20]:
bed_iou = []
closet_iou = []

In [21]:
for j in range(20):
    check = 'test_bed_left'
    if j <11:
        check = 'test_bed_left'
    else:
        check = 'test_bed_right'
    color_path = os.path.join(path, 'color_segmented/color_mask_{}.jpg'.format(j))
    depth_path = os.path.join(path, 'depth_segmented/depth_mask_{}.png'.format(j))
    cdp_masked = load_cdp(color_path, depth_path)
    
    if cdp_masked.color is None:
        bed_iou.append(0.0)
        closet_iou.append(0.0)
        pass
    else:

        icp_bed = MultiICP(model='model_CAD/bed/bed.STL',
                         Toff=SE3([[0,1,0],[0,0,1],[1,0,0]], [0.455,0,1.02]), scale=[1e-3,1e-3,1e-3])
        icp_bed.add_image(cdp_masked, Tc=None)


        Tguess = icp_bed.get_initial_by_center(R=np.matmul(T_cb[:3,:3], Rot_axis(3,np.pi)), 
                                                       offset=np.matmul(T_cb[:3,:3], (1.1*0.7,0,-0.6)))

        T_co1, fitness1 = icp_bed.compute_ICP(To=Tguess, thres=0.2, visualize=False)
        T_co2, fitness2 = icp_bed.compute_front_ICP(Tc_cur=np.identity(4), To=T_co1, thres=0.1, visualize=False)
        T_co_bed = T_co2
        T_bo_bed = np.matmul(T_bc, T_co2)

        bed = add_bed(gscene, T_bo_bed[:3,3], Rot2rpy(T_bo_bed[:3,:3]), (0,1,0,0.5))

        color_image = cv2.imread(os.path.join(path, 'color/color_{}.jpg'.format(j)))
        objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)

        T_co_gt = objectPose_dict[check]
        T_bo_gt = matmul_series(T_bc, T_co_gt)


        T_off = np.matmul(SE3_inv(T_bo_gt), T_bo_bed)
        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=(0,0,0))
        scale=[1e-3,1e-3,1e-3]

        bed_mesh = o3d.io.read_triangle_mesh('model_CAD/bed/bed.STL')
        bed_mesh_gt = o3d.io.read_triangle_mesh('model_CAD/bed/bed.STL')
        bed_mesh.vertices = o3d.utility.Vector3dVector(
                        np.asarray(bed_mesh.vertices) * np.array([scale[0], scale[1], scale[2]]))
        bed_mesh_gt.vertices = o3d.utility.Vector3dVector(
                        np.asarray(bed_mesh_gt.vertices) * np.array([scale[0], scale[1], scale[2]]))
        bed_mesh.transform(T_off)
        # mesh_gt.transform(T_co_gt)
        # aabox = mesh.get_oriented_bounding_box()
        # aabox_gt = mesh_gt.get_oriented_bounding_box()
        aabox = bed_mesh.get_axis_aligned_bounding_box()
        aabox_gt = bed_mesh_gt.get_axis_aligned_bounding_box()

    #     o3d.visualization.draw_geometries([bed_mesh, aabox, bed_mesh_gt, aabox_gt, origin])
        points = np.asarray(aabox.get_box_points())
        points_gt = np.asarray(aabox_gt.get_box_points())
        # points = points[points[:,2].argsort()[::-1]]
        # points_gt = points_gt[points_gt[:,2].argsort()[::-1]]

        #     iou_3d = get_3d_iou(aabox, aabox_gt, points, points_gt)
        iou_3d = get_iou_3d(aabox, aabox_gt, points, points_gt, T_off)
        bed_iou.append(iou_3d)


        check = 'test_closet_left'
        if j <11:
            check = 'test_closet_left'
        else:
            check = 'test_closet_right'

        CLOSET_LOCATION = "LEFT"
        color_image = cv2.imread(os.path.join(os.getcwd(), "save_img/bed/color/color_{}.jpg".format(j)), flags=cv2.IMREAD_UNCHANGED)
        depth_image = cv2.imread(os.path.join(os.getcwd(), "save_img/bed/depth/depth_{}.png".format(j)), flags=cv2.IMREAD_UNCHANGED)
        cdp = ColorDepthMap(color_image, depth_image, 
                        [1280, 720,
                         899.05322266, 899.21044922,
                         654.88366699, 352.92956543], 
                        0.000250000011874)

        icp_closet = MultiICP(model='model_CAD/top_table/top_table.STL', 
                              Toff=SE3([[1,0,0],[0,0,1],[0,-1,0]], [0.3,0,0.2725]))
        icp_closet.clear()

        pcd_closet = cdp2pcd(cdp, T_bc)

        pcd_masked = mask_boxes(pcd_closet, 
                            boxes=[gscene.NAME_DICT["bed_{}_space".format(CLOSET_LOCATION.lower())]], 
                            Q=VIEW_POSE_EXT, inside=True, 
                            merge_rule=np.all, link_ref="base_link")
        pcd_masked = mask_boxes(pcd_masked, 
                            boxes=[gscene.NAME_DICT["bed_box"], 
                                   gscene.NAME_DICT["bed_wall"]], 
                            Q=VIEW_POSE_EXT, inside=False, 
                            merge_rule=np.all, link_ref="base_link")
        pcd_masked, ind = pcd_masked.remove_radius_outlier(nb_points=20, radius=0.06)
        pcd_closet = icp_closet.add_pointcloud(pcd_masked, T_bc, ratio=0.5)

        bed_vis = gscene.NAME_DICT["bed_vis"]
        Tbs = bed_vis.get_tf(VIEW_POSE_EXT)
        initial_guess = icp_closet.get_initial_by_median(Tbs[:3, :3], np.matmul(T_bc[:3,:3], (0,0,0.3))-(0,0,1))
        T_bo_closet, fitness = icp_closet.compute_front_ICP(T_bc, initial_guess, thres=0.13, visualize=False)

        closet_leftup, closet_rightup, closet_down = add_closet(gscene,
                                    closet_center=T_bo_closet[:3,3], closet_rpy=Rot2rpy(T_bo_closet[:3,:3]),
                                    COLOR_CLOSET_COL=(0,1,0,0.3))

        z_offset = np.matmul(SE3_inv(T_bo_bed),T_bo_closet)[2,3]
        T_bo_closet= np.matmul(T_bo_closet,
                                SE3(np.identity(3), (0,0,-z_offset)))
        move_closet(gscene, closet_center=T_bo_closet[:3,3], closet_rpy=Rot2rpy(T_bo_closet[:3,:3]))


        objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)

        T_co_gt = objectPose_dict[check]
        T_bo_gt = matmul_series(T_bc, T_co_gt)


        T_off = np.matmul(SE3_inv(T_bo_gt), T_bo_closet)

        closet_mesh = o3d.io.read_triangle_mesh('model_CAD/top_table/top_table.STL')
        closet_mesh_gt = o3d.io.read_triangle_mesh('model_CAD/top_table/top_table.STL')
        closet_mesh.vertices = o3d.utility.Vector3dVector(
                        np.asarray(closet_mesh.vertices) * np.array([scale[0], scale[1], scale[2]]))
        closet_mesh_gt.vertices = o3d.utility.Vector3dVector(
                        np.asarray(closet_mesh_gt.vertices) * np.array([scale[0], scale[1], scale[2]]))
        closet_mesh.transform(T_off)
        # mesh_gt.transform(T_co_gt)
        # aabox = mesh.get_oriented_bounding_box()
        # aabox_gt = mesh_gt.get_oriented_bounding_box()
        aabox = closet_mesh.get_axis_aligned_bounding_box()
        aabox_gt = closet_mesh_gt.get_axis_aligned_bounding_box()

    #     o3d.visualization.draw_geometries([closet_mesh, aabox, closet_mesh_gt, aabox_gt, origin])
        points = np.asarray(aabox.get_box_points())
        points_gt = np.asarray(aabox_gt.get_box_points())
        # points = points[points[:,2].argsort()[::-1]]
        # points_gt = points_gt[points_gt[:,2].argsort()[::-1]]

        #     iou_3d = get_3d_iou(aabox, aabox_gt, points, points_gt)
        iou_3d = get_iou_3d(aabox, aabox_gt, points, points_gt, T_off)
        closet_iou.append(iou_3d)

Apply point-to-point ICP
registration::RegistrationResult with fitness=6.946462e-01, inlier_rmse=9.044456e-02, and correspondence_set size of 23627
Access transformation to get result.
Transformation is:
[[ 0.88382355 -0.1784018   0.43246817 -0.48831967]
 [-0.29439158 -0.93054209  0.21777284  0.229109  ]
 [ 0.36357877 -0.31978776 -0.87495501  4.34858179]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=5.256252e-01, inlier_rmse=4.021042e-02, and correspondence_set size of 8933
Access transformation to get result.
Transformation is:
[[ 0.90621671 -0.0638162   0.41796981 -0.48023212]
 [-0.16296234 -0.96488613  0.20600491  0.23407242]
 [ 0.39014682 -0.25479843 -0.88479558  4.35305778]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.832396447256611)
Apply point-to-point ICP
registration::RegistrationResult with fitness=5.036987e-01, inlier_rmse=6.197570e-02, and correspondence_set size of 8239
Acces

('IOU_3d:', 0.7085781768235577)
Apply point-to-point ICP
registration::RegistrationResult with fitness=7.475765e-01, inlier_rmse=4.964910e-02, and correspondence_set size of 7943
Access transformation to get result.
Transformation is:
[[-0.78917601  0.15097954 -0.59532043  4.35231905]
 [-0.61313091 -0.13739126  0.77794226  0.6791239 ]
 [ 0.03566153  0.97894273  0.2009961  -0.7266259 ]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.8372175123419494)
Apply point-to-point ICP
registration::RegistrationResult with fitness=7.229159e-01, inlier_rmse=9.530964e-02, and correspondence_set size of 9322
Access transformation to get result.
Transformation is:
[[ 0.54499726 -0.06651702  0.83579511 -1.1315662 ]
 [-0.25395287 -0.96311822  0.08894508  0.91742343]
 [ 0.79905315 -0.26072739 -0.54178898  4.293489  ]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=5.536767e-01, inlier_rmse=5.023159e-02, and corre

Access transformation to get result.
Transformation is:
[[ 0.79956682  0.17939968 -0.57315675 -0.50652983]
 [ 0.28975787 -0.9511557   0.10650447  0.47640337]
 [-0.52605445 -0.25123412 -0.81249501  4.65430969]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=7.039901e-01, inlier_rmse=3.991294e-02, and correspondence_set size of 10198
Access transformation to get result.
Transformation is:
[[ 0.8240669   0.09664638 -0.55818744 -0.55387399]
 [ 0.17298498 -0.98120699  0.08549296  0.60687789]
 [-0.53943483 -0.16700997 -0.82529857  4.68465835]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.8617408713146887)
Apply point-to-point ICP
registration::RegistrationResult with fitness=7.924442e-01, inlier_rmse=4.182934e-02, and correspondence_set size of 5181
Access transformation to get result.
Transformation is:
[[-0.82498158  0.17666414  0.53683816  4.61560283]
 [ 0.52173586 -0.12707144  0.84359023  0.741

Apply point-to-point ICP
registration::RegistrationResult with fitness=8.476190e-01, inlier_rmse=4.285603e-02, and correspondence_set size of 8989
Access transformation to get result.
Transformation is:
[[-8.22400084e-01  3.03464368e-01  4.81214578e-01  4.03535749e+00]
 [ 5.05041658e-01  4.09286252e-05  8.63094942e-01  3.98446301e-01]
 [ 2.61898872e-01  9.52842789e-01 -1.53295808e-01 -9.15140776e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
('IOU_3d:', 0.7811261273716289)


In [24]:
bed_iou

[0.832396447256611,
 0.7776322943631039,
 0.8065569691341592,
 0.8092657326081029,
 0.8722807461393363,
 0.852944480762681,
 0.7085781768235577,
 0.4721986578687814,
 0.7065536853959994,
 0.7084652792541528,
 0.863118635496137,
 0.8071994147882628,
 0.8566924008113863,
 0.8617408713146887,
 0.8600782448078845,
 0.8112365934923887,
 0.8830617958155006,
 0.8616161087303366,
 0.852993361101686,
 0.8690834174054392]

In [26]:
np.mean(np.array(bed_iou))

0.8029065829844514

In [25]:
np.savetxt('./exp_data/bed_iou_result_{}.txt'.format(confidence), np.array(bed_iou))

In [26]:
closet_iou

[0.7538778966023827,
 0.7470398910651865,
 0.7017440997064144,
 0.665677829854485,
 0.7662885438348125,
 0.8586250433827396,
 0.8372175123419494,
 0.3385392193973677,
 0.8115136627970831,
 0.4599290203429216,
 0.741014095925481,
 0.6122461055536358,
 0.5363710208807048,
 0.5379133958961622,
 0.6109714507628983,
 0.6283270334556137,
 0.6970632575485872,
 0.49114340162107595,
 0.6124969338077064,
 0.7811261273716289]

In [29]:
np.mean(np.array(closet_iou))

0.6573929952522963

In [27]:
np.savetxt('./exp_data/closet_iou_result_{}.txt'.format(confidence), np.array(closet_iou))

### Toilet

In [28]:
path = './save_img/toilet'

In [29]:
confidence = 0.01
obj_name_arg = obj_name_off + 'toilet'
img_num_arg = img_num_off + str(20)
confidence_arg = confidence_off + str(confidence)
result = subprocess.check_output(['python3',
                    os.getcwd() + '/test_seg_multi.py',
                    obj_name_arg, img_num_arg, confidence_arg])
result

'apex is not installed\napex is not installed\napex is not installed\napex is not installed\nUse load_from_local loader\nSegmentation Finish!\n'

In [30]:
toilet_iou = []

In [31]:
for j in range(20):
    color_path = os.path.join(path, 'color_segmented/color_mask_{}.jpg'.format(j))
    depth_path = os.path.join(path, 'depth_segmented/depth_mask_{}.png'.format(j))
    cdp_masked = load_cdp(color_path, depth_path)
    
    if cdp_masked.color is None:
        toilet_iou.append(0.0)
        pass
    else:
        icp_toilet = MultiICP(model='model_CAD/toilet_big.STL',
                         Toff=SE3(np.identity(3),(0,0,0)), scale=[1.24e-2,1e-2,1.05e-2])
        icp_toilet.add_image(cdp_masked, Tc=None)

        scale=[1.24e-2,1.23e-2,1.05e-2]
        toilet_model = o3d.io.read_triangle_mesh('model_CAD/toilet_big.STL')
        toilet_model.vertices = o3d.utility.Vector3dVector(
                        np.asarray(toilet_model.vertices) * np.array([scale[0], scale[1], scale[2]]))

        toilet_pcd = toilet_model.sample_points_uniformly(int(len(np.asarray(icp_toilet.pcd.points))*0.7))
        points = np.asarray(toilet_pcd.points)

        idx = []
        for i in range(len(points)):
            if points[i, 2] < 0.4:
                idx.append(i)

        pts = np.zeros((len(idx), 3))
        for i in range(len(idx)):
            pts[i] = points[idx[i]]

        cut_pcd = o3d.geometry.PointCloud()
        cut_pcd.points = o3d.utility.Vector3dVector(pts)
        icp_toilet.model_sampled = cut_pcd

        T_cb = SE3_inv(T_bc)
        Tguess = icp_toilet.get_initial_by_center(R=matmul_series(Rot_axis(1, -np.pi/2), T_cb[:3,:3], Rot_axis(3,np.pi/2)), 
                                                       offset=np.matmul(T_cb[:3,:3], (-0.2,-0.2,-0.3)))

        T_co1, fitness1 = icp_toilet.compute_ICP(To=Tguess, thres=0.13, visualize=False)
        T_bo_toilet=np.matmul(T_bc, T_co1)


        color_image = cv2.imread(os.path.join(path, 'color/color_{}.jpg'.format(j)))
        objectPose_dict, corner_dict = aruco_map.get_object_pose_dict(color_image, cameraMatrix, distCoeffs)

        T_co_gt = objectPose_dict['test_toilet']
        T_bo_gt = matmul_series(T_bc, T_co_gt)

        T_off = np.matmul(SE3_inv(T_bo_gt), T_bo_toilet)

        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=(0,0,0))
        toilet_mesh = o3d.io.read_triangle_mesh('model_CAD/toilet_big.STL')
        toilet_mesh_gt = o3d.io.read_triangle_mesh('model_CAD/toilet_big.STL')
        toilet_mesh.vertices = o3d.utility.Vector3dVector(
                        np.asarray(toilet_mesh.vertices) * np.array([scale[0], scale[1], scale[2]]))
        toilet_mesh_gt.vertices = o3d.utility.Vector3dVector(
                        np.asarray(toilet_mesh_gt.vertices) * np.array([scale[0], scale[1], scale[2]]))
        toilet_mesh.transform(T_off)
        # mesh_gt.transform(T_co_gt)
        # aabox = mesh.get_oriented_bounding_box()
        # aabox_gt = mesh_gt.get_oriented_bounding_box()
        aabox = toilet_mesh.get_axis_aligned_bounding_box()
        aabox_gt = toilet_mesh_gt.get_axis_aligned_bounding_box()

    #     o3d.visualization.draw_geometries([mesh, aabox, mesh_gt, aabox_gt, origin])
        points = np.asarray(aabox.get_box_points())
        points_gt = np.asarray(aabox_gt.get_box_points())
        # points = points[points[:,2].argsort()[::-1]]
        # points_gt = points_gt[points_gt[:,2].argsort()[::-1]]

    #     iou_3d = get_3d_iou(aabox, aabox_gt, points, points_gt)
        iou_3d = get_iou_3d(aabox, aabox_gt, points, points_gt, T_off)
        toilet_iou.append(iou_3d)

Apply point-to-point ICP
registration::RegistrationResult with fitness=9.796713e-01, inlier_rmse=4.885253e-02, and correspondence_set size of 24144
Access transformation to get result.
Transformation is:
[[-0.93342555 -0.08632043 -0.348232    0.34196324]
 [ 0.15937723 -0.96936072 -0.18691893  0.37135426]
 [-0.3214275  -0.22997515  0.91858358  1.50925379]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.6720283022252945)
Apply point-to-point ICP
registration::RegistrationResult with fitness=9.929815e-01, inlier_rmse=4.571806e-02, and correspondence_set size of 40322
Access transformation to get result.
Transformation is:
[[-0.99636112  0.01956389 -0.08295641  0.38789604]
 [ 0.01236159 -0.92983935 -0.36775803  0.40863237]
 [-0.08433091 -0.36744528  0.92621394  1.37720103]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.694226609945278)
Apply point-to-point ICP
registration::RegistrationResult with fitness=9.912069e-01, inlier_rmse=4.736227e-02, and corr

Access transformation to get result.
Transformation is:
[[-0.91724783 -0.08143733 -0.38990304  0.50004092]
 [ 0.1423604  -0.98124684 -0.12995447  0.32519208]
 [-0.37200798 -0.17470721  0.91163998  1.16511994]
 [ 0.          0.          0.          1.        ]]
('IOU_3d:', 0.6142496361474816)


In [33]:
toilet_iou

[0.6720283022252945,
 0.694226609945278,
 0.7274802992934615,
 0.6455283954778708,
 0.5770543651503316,
 0.46542363105626355,
 0.5089238181834036,
 0.625887616803927,
 0.6491463606628801,
 0.5966616854589674,
 0.6768684508879987,
 0.6634442588213447,
 0.6840515577245821,
 0.5987026974145231,
 0.590344149943627,
 0.6814838347605233,
 0.6485348056637381,
 0.5675581471488402,
 0.5508181147887905,
 0.6142496361474816]

In [45]:
np.mean(np.array(toilet_iou))

0.6248112154827152

In [34]:
np.savetxt('./exp_data/toilet_iou_result_{}.txt'.format(confidence), np.array(toilet_iou))